In [1]:
import pandas as pd
import numpy as np
from datetime import date,time,timedelta

path_cases = 'https://moh-malaysia-covid19.s3.ap-southeast-1.amazonaws.com/linelist_cases.parquet'
path_param_geo = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/linelist/param_geo.csv'


In [2]:
geo = pd.read_csv(path_param_geo)
idxd_district = dict(zip(geo.idxd, geo.district))
idxs_state = dict(zip(geo.idxs, geo.state))
district_state = dict(zip(geo.district, geo.state))

df = pd.read_parquet(path_cases, columns=['date', 'state', 'district'])
df['state'] = df.state.map(idxs_state)
df['district'] = df.district.map(idxd_district)
df.head()

,date,state,district
0,2020-01-25,Johor,Johor Bahru
1,2020-01-25,Johor,Johor Bahru
2,2020-01-25,Johor,Johor Bahru
3,2020-01-25,Johor,Johor Bahru
4,2020-01-29,Johor,Johor Bahru


In [3]:
df = df[~df.district.isnull()].copy()
df.date = pd.to_datetime(df.date)
df['cases'] = 1

df = df.groupby(['date','district'])\
    .sum()\
    .unstack(fill_value=0)\
    .asfreq('D', fill_value=0)\
    .stack()\
    .sort_index(level=1)\
    .reset_index()
df.date = df.date.dt.date
df['state'] = df.district.map(district_state)
df = df[['date','state','district','cases']]
df = df.sort_values(by=['date','state','district']).reset_index(drop=True)
df.head()

,date,state,district,cases
0,2020-01-25,Johor,Batu Pahat,0
1,2020-01-25,Johor,Johor Bahru,4
2,2020-01-25,Johor,Kluang,0
3,2020-01-25,Johor,Kota Tinggi,0
4,2020-01-25,Johor,Kulai,0
